# Voxel pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher
from VoxelGrid import VoxelGrid

%matplotlib ipympl

### Load calibrations and other data

In [2]:
imageProcessor = ImageProcessor()
imageProcessor.loadMonoCalibrationResults()
imageProcessor.loadStereoCalibration()
imageProcessor.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [3]:
stereoMatcher = StereoMatcher(imageProcessor=imageProcessor, \
                matcher="SGBM", vertical=True, createRightMatcher=False)

imageProcessor.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Loading images

In [4]:
path = "testImages/voxelTestImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-9


### Select image pair and display

In [5]:
imageNumber = 1

imageL = cv2.imread(imageGlobL[imageNumber])
imageR = cv2.imread(imageGlobR[imageNumber])

plt.figure()
plt.imshow(cv2.cvtColor(np.hstack([imageL, imageL]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Convert to grayscale and undistort

In [6]:
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

### View undistorted image

In [7]:
fig = plt.figure()
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig = plt.figure()
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig = plt.figure(figsize=(6,15))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [10]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 5.0
maxDisparity: 37.0


### View disparity map

In [11]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE), cmap="gray")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [12]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

5.0
37.0
(640, 360)
533.37445
3946.9712


### View depth map

In [13]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE), cmap="Greys")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# Use to save images
# _=cv2.imwrite("depth.png", cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

### Compute point cloud

In [15]:
voxelGrid = VoxelGrid(stereoMatcher, imageProcessor)
voxelGrid.verbose = True

In [16]:
voxelGrid.generatePointCloud()

Points in unfiltered pointcloud: 11520; completed in 0.02043 sec


Filtering extreme points

In [17]:
voxelGrid.filterPointCloud()

Points in filtered pointcloud: 8706; completed in 0.00590 sec


Rotate point cloud to have y forward

In [18]:
voxelGrid.redefinePointCloudCoordinate()

### View point cloud

In [19]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.pointCloud[:,0], voxelGrid.pointCloud[:,1], \
    voxelGrid.pointCloud[:,2], s=1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_xlim(-1000,1000)
ax.set_ylim(0,2000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [20]:
voxelGrid.resetVoxelGrid()
voxelGrid.voxelizePointCloud()

Voxel grid reset
Voxels in grid: 94; completed in 0.14317 sec; 115 iterations


### View voxelized point cloud

In [21]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection = "3d")

ax.scatter(voxelGrid.voxelGrid[:,0], \
        voxelGrid.voxelGrid[:,1], \
        voxelGrid.voxelGrid[:,2])

ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")

# Camera axis begins at 0
ax.set_xlim(-1000,1000)
ax.set_ylim(0,2000)
ax.set_zlim(-1000,1000)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …